In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from matplotlib import pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

import pickle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import os

import string

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_addons as tfa
from numpy import dstack
from sklearn.linear_model import LogisticRegression


In [2]:
df = pd.read_table('/kaggle/input/fb-post-data/FB_posts_labeled.txt')

In [3]:
df['text'] = df['message'].map(lambda text: text.translate(str.maketrans('','',string.punctuation)).strip())
df['text'] = df['text'].map(lambda text: text.translate(str.maketrans('','',string.digits)).strip())

In [4]:
##Only use while reading 1 dataset
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df[['Appreciation', 'Complaint', 'Feedback']], 
                                                                    random_state=69, 
                                                                    test_size=0.3, 
                                                                    stratify=df[['Appreciation', 'Complaint', 'Feedback']])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=69, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [5]:
print(train_labels.sum(axis = 0) / train_labels.shape[0])
print(val_labels.sum(axis = 0) / val_labels.shape[0])
print(test_labels.sum(axis = 0) / test_labels.shape[0])

Appreciation    0.258973
Complaint       0.534458
Feedback        0.206569
dtype: float64
Appreciation    0.258794
Complaint       0.534338
Feedback        0.206868
dtype: float64
Appreciation    0.259414
Complaint       0.534728
Feedback        0.205858
dtype: float64


In [7]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=744cc359333b07dc8e6c432e0f4d6317bf30f2311a1aeabe015bfa87bc48bcce
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

### Model building

In [10]:
embeddings_train = model.encode(train_text.to_list())
embeddings_val = model.encode(val_text.to_list())
embeddings_test = model.encode(test_text.to_list())

Batches:   0%|          | 0/175 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [11]:
embeddings_train.shape

(5572, 768)

In [16]:
f1 = tfa.metrics.F1Score(num_classes = 3, average = 'macro')

callbacks = [
    tf.keras.callbacks.EarlyStopping(     #EarlyStopping is used to stop at the epoch where val_accuracy does not improve significantly
        monitor='val_loss',
        min_delta=1e-4,
        patience=5,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='weights_use.h5',
        monitor='val_loss', 
        mode='min', 
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

In [19]:
def SBERT_model_builder():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape = (768,), dtype = 'float32'))
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)))
    #model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics= [f1])
    
    return model

In [20]:
sbert_model = SBERT_model_builder()
sbert_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               196864    
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 197,635
Trainable params: 197,635
Non-trainable params: 0
_________________________________________________________________


In [22]:
sbert_model.fit(embeddings_train, 
          train_labels, 
          epochs=30, 
          callbacks=callbacks,
          batch_size = 8,
          validation_data= (embeddings_val, val_labels))

Epoch 1/30
693/697 [============================>.] - ETA: 0s - loss: 0.8474 - f1_score: 0.7379
Epoch 1: val_loss improved from inf to 0.60967, saving model to weights_use.h5
697/697 [==============================] - 5s 4ms/step - loss: 0.8459 - f1_score: 0.7386 - val_loss: 0.6097 - val_f1_score: 0.8005
Epoch 2/30
692/697 [============================>.] - ETA: 0s - loss: 0.6176 - f1_score: 0.7893
Epoch 2: val_loss improved from 0.60967 to 0.59617, saving model to weights_use.h5
697/697 [==============================] - 3s 4ms/step - loss: 0.6163 - f1_score: 0.7900 - val_loss: 0.5962 - val_f1_score: 0.8115
Epoch 3/30
683/697 [============================>.] - ETA: 0s - loss: 0.6066 - f1_score: 0.7930
Epoch 3: val_loss did not improve from 0.59617
697/697 [==============================] - 3s 4ms/step - loss: 0.6077 - f1_score: 0.7918 - val_loss: 0.6295 - val_f1_score: 0.7850
Epoch 4/30
693/697 [============================>.] - ETA: 0s - loss: 0.5960 - f1_score: 0.7874
Epoch 4: val_l

In [24]:
sbert_model.evaluate(embeddings_test, test_labels)

38/38 [==============================] - 0s 3ms/step - loss: 0.5220 - f1_score: 0.7982


[0.5219564437866211, 0.7982286810874939]

### HP tuning

In [10]:
pip install -q -U keras-tuner

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires

In [11]:
import keras_tuner as kt

In [14]:
f1 = tfa.metrics.F1Score(num_classes = 3, average = 'macro')

def sbert_model_builder_HP(hp):
    model = tf.keras.models.Sequential()
    
    #Input layer
    model.add(tf.keras.layers.Input(shape = (768,), dtype = 'float32'))
    
    #Layer 1
    hp_units = hp.Int('units', min_value=192, max_value=512, step=32)
    hp_l2 = hp.Choice('l2', values=[1e-1, 1e-2, 1e-3, 1e-4])
    model.add(tf.keras.layers.Dense(units = hp_units, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(hp_l2)))
    
    #Layer 2
    model.add(tf.keras.layers.Dense(units = hp_units, activation='relu'))
    
    #Layer 3
    hp_dropout = hp.Choice('Dropout', values=[0.0, 0.05,0.1])
    model.add(tf.keras.layers.Dropout(hp_dropout))
    
    # Output layer
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) 
    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='categorical_crossentropy',
                metrics=[f1])
    
    return model

In [16]:
tuner = kt.Hyperband(sbert_model_builder_HP,
                     objective= kt.Objective("val_f1_score", direction="max"),
                     max_epochs=50,
                     factor=3)

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=5)

tuner.search(embeddings_train, train_labels,
             epochs= 25,
             validation_data=(embeddings_val, val_labels), 
             callbacks=[stop_early])

Trial 90 Complete [00h 00m 12s]
val_f1_score: 0.8320834040641785

Best val_f1_score So Far: 0.8534145355224609
Total elapsed time: 00h 10m 26s


In [17]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

sbert_tuned_model = tuner.hypermodel.build(best_hps)
history = sbert_tuned_model.fit(embeddings_train, train_labels, epochs=50, validation_data=(embeddings_val, val_labels))

val_f1_per_epoch = history.history['val_f1_score']
best_epoch = val_f1_per_epoch.index(max(val_f1_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
175/175 [==============================] - 2s 5ms/step - loss: 0.5235 - f1_score: 0.7899 - val_loss: 0.4340 - val_f1_score: 0.8338
Epoch 2/50
175/175 [==============================] - 1s 4ms/step - loss: 0.3879 - f1_score: 0.8545 - val_loss: 0.4424 - val_f1_score: 0.8368
Epoch 3/50
175/175 [==============================] - 1s 4ms/step - loss: 0.3300 - f1_score: 0.8860 - val_loss: 0.4340 - val_f1_score: 0.8509
Epoch 4/50
175/175 [==============================] - 1s 4ms/step - loss: 0.2697 - f1_score: 0.9116 - val_loss: 0.4653 - val_f1_score: 0.8439
Epoch 5/50
175/175 [==============================] - 1s 4ms/step - loss: 0.2151 - f1_score: 0.9391 - val_loss: 0.5104 - val_f1_score: 0.8452
Epoch 6/50
175/175 [==============================] - 1s 4ms/step - loss: 0.1600 - f1_score: 0.9653 - val_loss: 0.6048 - val_f1_score: 0.8404
Epoch 7/50
175/175 [==============================] - 1s 4ms/step - loss: 0.1301 - f1_score: 0.9773 - val_loss: 0.6521 - val_f1_score: 0.8257
Epoch 

In [18]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(embeddings_train, train_labels, epochs=best_epoch, validation_data=(embeddings_val, val_labels))

Epoch 1/3
175/175 [==============================] - 2s 5ms/step - loss: 0.5311 - f1_score: 0.7923 - val_loss: 0.4344 - val_f1_score: 0.8404
Epoch 2/3
175/175 [==============================] - 1s 4ms/step - loss: 0.3837 - f1_score: 0.8554 - val_loss: 0.4337 - val_f1_score: 0.8377
Epoch 3/3
175/175 [==============================] - 1s 4ms/step - loss: 0.3246 - f1_score: 0.8869 - val_loss: 0.4316 - val_f1_score: 0.8389


In [19]:
eval_result = hypermodel.evaluate(embeddings_test, test_labels)
print("[test loss, test f1]:", eval_result)

38/38 [==============================] - 0s 3ms/step - loss: 0.4227 - f1_score: 0.8370
[test loss, test f1]: [0.42269405722618103, 0.8369593620300293]


In [20]:
hypermodel.save('fb_hyper_sbert.hdf5')

### Stacking model

In [10]:
f1 = tfa.metrics.F1Score(num_classes = 3, average = 'macro')

In [9]:
embeddings_val = model.encode(val_text.to_list())
embeddings_test = model.encode(test_text.to_list())

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [75]:
print(embeddings_val.shape)
print(embeddings_test.shape)

(1194, 768)
(1195, 768)


In [80]:
all_embeds = np.append(embeddings_val, embeddings_test, axis = 0)

In [81]:
all_embeds.shape

(2389, 768)

In [93]:
all_labels = pd.concat([val_labels, test_labels], axis = 0)

In [95]:
inputy = all_labels.idxmax(1)

In [102]:
all_text = pd.concat([val_text, test_text], axis = 0)

In [104]:
all_text.shape

(2389,)

In [13]:
KerasLayer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder-large/5', 
                        input_shape=[], 
                        dtype=tf.string, 
                        trainable=False)

In [97]:
# load models from file
def load_all_models():
    all_models = list()
    model_names = ['/kaggle/input/models-for-stacking/fb_hyper_sbert.hdf5',
                   '/kaggle/input/models-for-stacking/fb_hyper_use_LARGE.hdf5',
                   '/kaggle/input/models-for-stacking/fb_hyper_use_large_class.hdf5']
    for i in range(len(model_names)):
        # load models
        model = tf.keras.models.load_model(model_names[i],custom_objects={'KerasLayer': KerasLayer,
                                                          'f1' : f1})
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % model_names[i])
    return all_models

In [98]:
members = load_all_models()
print('Loaded %d models' % len(members))

>loaded /kaggle/input/models-for-stacking/fb_hyper_sbert.hdf5
>loaded /kaggle/input/models-for-stacking/fb_hyper_use_LARGE.hdf5
>loaded /kaggle/input/models-for-stacking/fb_hyper_use_large_class.hdf5
Loaded 3 models


In [16]:
members[0].summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 448)               344512    
                                                                 
 dense_7 (Dense)             (None, 448)               201152    
                                                                 
 dropout_2 (Dropout)         (None, 448)               0         
                                                                 
 dense_8 (Dense)             (None, 3)                 1347      
                                                                 
Total params: 547,011
Trainable params: 547,011
Non-trainable params: 0
_________________________________________________________________


In [17]:
members[1].summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 512)               147354880 
                                                                 
 dense_6 (Dense)             (None, 384)               196992    
                                                                 
 dense_7 (Dense)             (None, 384)               147840    
                                                                 
 dropout_2 (Dropout)         (None, 384)               0         
                                                                 
 dense_8 (Dense)             (None, 3)                 1155      
                                                                 
Total params: 147,700,867
Trainable params: 345,987
Non-trainable params: 147,354,880
_________________________________________________________________


In [18]:
members[2].summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 512)               147354880 
                                                                 
 dense_6 (Dense)             (None, 448)               229824    
                                                                 
 dense_7 (Dense)             (None, 448)               201152    
                                                                 
 dropout_2 (Dropout)         (None, 448)               0         
                                                                 
 dense_8 (Dense)             (None, 3)                 1347      
                                                                 
Total params: 147,787,203
Trainable params: 432,323
Non-trainable params: 147,354,880
_________________________________________________________________


In [99]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX_sbert, inputX_use):
    stackX = None
    # make prediction
    yhat0 = members[0].predict(inputX_sbert, verbose=0)
    # stack predictions into [rows, members, probabilities]
    stackX = yhat0 #

    yhat1 = members[1].predict(inputX_use, verbose=0)
    stackX = dstack((stackX, yhat1))
    
    yhat2 = members[2].predict(inputX_use, verbose=0)
    stackX = dstack((stackX, yhat2))
    
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

In [105]:
test_stack = stacked_dataset(members, all_embeds, all_text)

In [106]:
test_stack.shape

(2389, 9)

In [108]:
labels = {'Appreciation': 0,
             'Complaint': 1, 
             'Feedback': 2}

inputy = inputy.map(labels) # re-mapping

In [109]:
inputy

5905    1
1839    1
6975    1
865     0
4194    0
       ..
1235    2
4336    2
6882    0
6418    2
7526    1
Length: 2389, dtype: int64

In [110]:
#import xgboost as xgb

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX_sbert, inputX_use, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX_sbert, inputX_use)
    # fit standalone model
#     model = xgb.XGBClassifier(objective='multi:softmax', 
#                             num_class=3) #meta learner
    model = LogisticRegression(multi_class = "ovr", class_weight ={0: 1, 1 : 1, 2: 3})
    model.fit(stackedX, inputy)
    return model

In [112]:
stacked_model = fit_stacked_model(members, all_embeds, all_text, inputy)

In [113]:
import pickle
filename = 'stacked_model_log_v3.sav'
pickle.dump(stacked_model, open(filename, 'wb'))
#stacked_model = pickle.load(open(filename, 'rb'))

In [114]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX_sbert, inputX_use):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX_sbert, inputX_use)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [52]:
yhat = stacked_prediction(members, stacked_model, embeddings_test, test_text)

In [55]:
#Only for xgb predictions
yhat = pd.Series(yhat).map({0 : 'Appreciation',
                     1 : 'Complaint', 
                     2 : 'Feedback'})

In [56]:
yhat = pd.get_dummies(yhat)

In [58]:
print(classification_report(test_labels, yhat))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90       310
           1       0.89      0.90      0.90       639
           2       0.83      0.81      0.82       246

   micro avg       0.88      0.88      0.88      1195
   macro avg       0.88      0.87      0.87      1195
weighted avg       0.88      0.88      0.88      1195
 samples avg       0.88      0.88      0.88      1195



In [ ]:
# loaded_model = tf.keras.models.load_model('/kaggle/input/use-hyper-tuned-model/USE_hyper.hdf5', custom_objects={'KerasLayer': KerasLayer})

### Adding predictions

In [59]:
predict_df = pd.read_table('/kaggle/input/fb-post-data/FB_posts_unlabeled.txt')

In [60]:
predict_df.head()

,postId,message
0,108381603303_10151119973393304,Love. It. To
1,115568331790246_371841206162956,NICE
2,115568331790246_515044031842672,Congrats
3,147285781446_10151010892176447,Awesome!
4,159616034235_10150639103634236,Award


In [61]:
predict_df['text'] = predict_df['message'].map(lambda text: text.translate(str.maketrans('','',string.punctuation)).strip())
predict_df['text'] = predict_df['text'].map(lambda text: text.translate(str.maketrans('','',string.digits)).strip())

In [33]:
embeddings_pred = model.encode(predict_df['text'].to_list())

Batches:   0%|          | 0/64 [00:00<?, ?it/s]

In [115]:
yhat = stacked_prediction(members, stacked_model, embeddings_pred, predict_df['text'])

In [116]:
raw_preds = pd.get_dummies(yhat)

In [117]:
raw_preds.sum(axis = 0)

0     548
1    1048
2     443
dtype: int64

In [118]:
raw_preds

,0,1,2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
2034,0,1,0
2035,0,1,0
2036,0,1,0
2037,0,1,0


In [119]:
preds_df = pd.concat([predict_df['text'], raw_preds], axis = 1)

In [120]:
post_ids = np.array(predict_df['postId'])

In [121]:
# Don't run before adaptation, this is pseudocode!
f = open('predictions_stacked_log_v3.csv', 'w')
# write the first header row
f.write('postId,Appreciation_pred,Complaint_pred,Feedback_pred' + '\n')

for i in range(post_ids.size):
    Appreciation_pred, Complaint_pred, Feedback_pred = raw_preds.iloc[i,0],  raw_preds.iloc[i,1],  raw_preds.iloc[i,2]
    if Appreciation_pred not in [0,1] or Complaint_pred not in [0,1] or Feedback_pred not in [0,1]:
        print('did you forget to turn probability predictions into binary predictions?')
    if Appreciation_pred + Complaint_pred + Feedback_pred != 1:
        print('SOMETHING_IS_WRONG')
    f.write(post_ids[i] + ',' + str(Appreciation_pred) + ',' + str(Complaint_pred) + ',' + str(Feedback_pred) + '\n')
f.close()

### Predictions for single models

In [79]:
use_model_hyper = tf.keras.models.load_model('/kaggle/input/stacking-models-fb/fb_hyper_use.hdf5', custom_objects={'KerasLayer': KerasLayer, 'f1' : f1})

In [80]:
use_model_hyper.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 512)               147354880 
                                                                 
 dense_6 (Dense)             (None, 384)               196992    
                                                                 
 dense_7 (Dense)             (None, 384)               147840    
                                                                 
 dropout_2 (Dropout)         (None, 384)               0         
                                                                 
 dense_8 (Dense)             (None, 3)                 1155      
                                                                 
Total params: 147,700,867
Trainable params: 345,987
Non-trainable params: 147,354,880
_________________________________________________________________


In [81]:
raw_preds = use_model_hyper.predict(predict_df['text'])

64/64 [==============================] - 10s 81ms/step


In [82]:
from numpy import argmax

In [83]:
raw_preds

array([[9.9297792e-01, 6.2972913e-03, 7.2476501e-04],
       [9.8801893e-01, 1.0550145e-02, 1.4309115e-03],
       [9.9461645e-01, 4.6169120e-03, 7.6668570e-04],
       ...,
       [1.5306695e-01, 5.6700450e-01, 2.7992862e-01],
       [5.7534602e-02, 9.3357289e-01, 8.8926041e-03],
       [2.5727996e-01, 7.3755085e-01, 5.1692054e-03]], dtype=float32)

In [84]:
for record in raw_preds:
    max_index =argmax(record)
    for i in range(3):
        if i == max_index:
            record[i] = 1
        else:
            record[i] = 0

In [85]:
post_ids = np.array(predict_df['postId'])

In [86]:
raw_preds = raw_preds.astype('int')

In [87]:
raw_preds

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [88]:
# Don't run before adaptation, this is pseudocode!
f = open('predictions_USE_hyper.csv', 'w')
# write the first header row
f.write('postId,Appreciation_pred,Complaint_pred,Feedback_pred' + '\n')

for i in range(post_ids.size):
    Appreciation_pred, Complaint_pred, Feedback_pred = raw_preds[i][0], raw_preds[i][1], raw_preds[i][2]
    if Appreciation_pred not in [0,1] or Complaint_pred not in [0,1] or Feedback_pred not in [0,1]:
        print('did you forget to turn probability predictions into binary predictions?')
    if Appreciation_pred + Complaint_pred + Feedback_pred != 1:
        print('SOMETHING_IS_WRONG')
    f.write(post_ids[i] + ',' + str(Appreciation_pred) + ',' + str(Complaint_pred) + ',' + str(Feedback_pred) + '\n')
f.close()

### Loading model